In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json
from datetime import datetime

# importing csv files.
crimes_19 = pd.read_csv('input_data/Chicago_Crimes_2019.csv')
crimes_20 = pd.read_csv('input_data/Chicago_Crimes_2020.csv')
crimes_21 = pd.read_csv('input_data/Chicago_Crimes_2021.csv')

# merging crimes data for the last 3 years into a signle dataframe
crimes_data = pd.merge((pd.merge(crimes_19,crimes_20, how="outer")),crimes_21,how="outer")

print(len(crimes_data))

679438


In [12]:
# converting Date column in dataframe to date format and adding columns for month, year, day, time
crimes_data["Date"] = pd.to_datetime(crimes_data['Date'])
crimes_data["Day"] = crimes_data['Date'].dt.day_name()
crimes_data["Month Day"] =crimes_data['Date'].dt.day
crimes_data["Month"] = crimes_data['Date'].dt.month_name()
crimes_data["Year"] = crimes_data['Date'].dt.year
crimes_data["Time"] =crimes_data['Date'].dt.time
crimes_data["AM_PM"] = pd.to_datetime(crimes_data['Date']).dt.strftime("%p")
print(crimes_data.shape)
crimes_data.dropna(how = 'any', inplace = True)
print(crimes_data.shape)
#crimes_data

(679438, 29)
(667920, 29)


In [6]:
crimes_data.columns

Index(['ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Primary Type',
       'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat',
       'District', 'District_name', 'District_population', 'Ward',
       'Community Area', 'FBI Code', 'X Coordinate', 'Y Coordinate',
       'Updated On', 'Latitude', 'Longitude', 'Location', 'Day', 'Month Day',
       'Month', 'Year', 'Time', 'AM_PM'],
      dtype='object')